In [2]:
!pip install pytorch-metric-learning
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 111 kB 14.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 141 kB/s 


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
from torchvision import datasets, transforms

from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator


### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.ln = nn.Linear(8640, 128) #should change

    def forward(self, x):
        x = self.ln(x)
        return x


### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
def train(model, loss_func, mining_func, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(data)
        indices_tuple = mining_func(embeddings, labels)
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(
                "Epoch {} Iteration {}: Loss = {}, Number of mined triplets = {}".format(
                    epoch, batch_idx, loss, mining_func.num_triplets
                )
            )


### convenient function from pytorch-metric-learning ###
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)


### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, train_embeddings, test_labels, train_labels, False
    )
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import numpy as np
with open('/content/drive/MyDrive/uci/walking_labels.npz','rb') as f:
  labels = np.load(f)['arr_0']
train_mask = labels<=20

In [9]:
class myDataset(torch.utils.data.Dataset):
  def __init__(self, features, labels):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.features = features
        self.labels = labels

  def __len__(self):
      return self.labels.shape[0]

  def __getitem__(self, idx):
      return features[idx], labels[idx]


device = torch.device("cuda")

batch_size = 16



In [13]:
import numpy as np
features = np.load('/content/drive/MyDrive/uci/walking_data_limu_bert.npz')['arr_0']
train_l = labels[train_mask]
train_f = features[train_mask]
test_l = labels[np.logical_not(train_mask)]
test_f = features[np.logical_not(train_mask)]

dataset1 = myDataset(train_f, train_l)
dataset2 = myDataset(test_f, test_l) #
train_loader = torch.utils.data.DataLoader(
    dataset1, batch_size=batch_size, shuffle=True
)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=batch_size)

model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
num_epochs = 10

In [14]:
### pytorch-metric-learning stuff ###
distance = distances.LpDistance()
reducer = reducers.ThresholdReducer(low=0)
loss_func = losses.TripletMarginLoss(margin=0.2, distance=distance, reducer=reducer)
mining_func = miners.TripletMarginMiner(
    margin=0.2, distance=distance, type_of_triplets="semihard"
)
accuracy_calculator = AccuracyCalculator(include=("precision_at_1",), k=1)
### pytorch-metric-learning stuff ###


for epoch in range(1, num_epochs + 1):
    train(model, loss_func, mining_func, device, train_loader, optimizer, epoch)
    test(dataset1, dataset2, model, accuracy_calculator)

Epoch 1 Iteration 0: Loss = 0.10746381431818008, Number of mined triplets = 42


100%|██████████| 4/4 [00:00<00:00, 31.69it/s]


Computing accuracy
Test set accuracy (Precision@1) = 1.0
Epoch 2 Iteration 0: Loss = 0.10504515469074249, Number of mined triplets = 19


100%|██████████| 4/4 [00:00<00:00, 34.82it/s]


Computing accuracy
Test set accuracy (Precision@1) = 1.0
Epoch 3 Iteration 0: Loss = 0.1029612123966217, Number of mined triplets = 23


...

Computing accuracy
Test set accuracy (Precision@1) = 1.0
Epoch 9 Iteration 0: Loss = 0.10460007190704346, Number of mined triplets = 18


100%|██████████| 4/4 [00:00<00:00, 31.26it/s]


Computing accuracy
Test set accuracy (Precision@1) = 1.0
Epoch 10 Iteration 0: Loss = 0.0787685364484787, Number of mined triplets = 10


100%|██████████| 4/4 [00:00<00:00, 32.88it/s]


Computing accuracy
Test set accuracy (Precision@1) = 1.0


In [25]:
from sklearn.metrics import top_k_accuracy_score

model.eval()
def eval_emb():
  y_pred = []
  y_true = []
  for i in range(test_l.shape[0]):
    trial = []
    for j in range(10):
      cl=j+21
      cl_samples = test_f[test_l==cl]
      trial.append(cl_samples[np.random.choice(cl_samples.shape[0])])
    x = test_f[i]
    # trial = np.concatenate(trial, axis=0)
    trial = np.vstack(trial)
    trial = np.concatenate((trial, x[None,:]), axis=0)
    with torch.no_grad():
      embed = model(torch.tensor(trial).to(device))
      embed = embed.cpu().numpy()
    trial = embed[:-1,:]
    x = embed[-1,:]
    # print('trual shape is',trial.shape)
    scores = -np.mean((trial - x)**2, axis=-1)
    # print('scores shape is',scores.shape)
    y_pred.append(scores)
    y_true.append(test_l[i])
  y_true = np.asarray(y_true)
  y_pred = np.vstack(y_pred)
  # print('y_pred shape', y_pred.shape)
  # print('y_true shape', y_true.shape)
  t1 = top_k_accuracy_score(y_true, y_pred,k=1, labels = list(range(21,31)), normalize=True)
  t3 = top_k_accuracy_score(y_true, y_pred,k=3, labels = list(range(21,31)), normalize=True)
  t5 = top_k_accuracy_score(y_true, y_pred,k=5, labels = list(range(21,31)), normalize=True)
  print('top 1 accuracy is',t1)
  print('top 3 accuracy is', t3)
  print('top 5 accuracy is', t5)
  # print(y_pred)
  # print(y_true)

eval_emb()

top 1 accuracy is 0.3418803418803419
top 3 accuracy is 0.49572649572649574
top 5 accuracy is 0.6239316239316239
